In [26]:
%reload_ext autoreload
%autoreload 2



In [27]:
#クリーニングしたwebテキストをクラスタリングするモデルを作る
n_clusters=30

In [28]:
from src.cleaner.auto_cleaner import clean_text
from datasets import load_dataset

In [29]:
txt="こんにちは､げんきですか?研究を "
clean_text(txt)

'こんにちは、げんきですか?研究を'

In [30]:
import glob
import dask.dataframe as dd

# localに落したc4の読み込み例｡｡｡

c4_path=glob.glob("../data/original_dump/c4/multilingual/c4-ja.*.json.gz")
for path in c4_path:
    try:
        df = dd.read_json(path)
        print("success")
        break
    except:
        continue


import gzip
import json

def read_gzip_json_file(file_path):
    """GZIP圧縮されたJSONファイルから順にテキストデータを読み込むイテレータを生成します。"""
    with gzip.open(file_path, 'rt', encoding='utf-8') as f:
        for line in f:
            yield json.loads(line)

lines=[]
for article in read_gzip_json_file(path):
    text = article.get('text', '')  # 'text'キーからテキストデータを取得
    lines.append(text)

In [ ]:
mc4_dataset=[{"text":text} for text in lines]

#mc4の読み込み
dataset_list = [
    load_dataset('mc4', 'ja',split='train', streaming=True),
    #mc4_dataset,
    load_dataset('oscar', 'unshuffled_deduplicated_ja',
                        split='train',streaming=True),
   load_dataset('cc100', 'ja', split='train',streaming=True),
   load_dataset("augmxnt/shisa-pretrain-en-ja-v1",
                        split='train',streaming=True),
]



/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for oscar contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/oscar
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(
/home/hatakeyama/miniconda3/envs/textprocess/lib/python3.12/site-packages/datasets/load.py:1461: FutureWarning: The repository for cc100 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/cc100
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.wa

In [ ]:
from tqdm import tqdm

#各データセットごと､1万件のデータを取得
max_articles=20000
cleaned_text=[]

for dataset in dataset_list:
    cnt=0
    dataset_iter=iter(dataset)

    for i in tqdm(range(max_articles*10)):
        text=next(dataset_iter)["text"]
        text=clean_text(text)

        if text!="":
            cleaned_text.append(text)
            cnt+=1

        if cnt>max_articles:
            break

  0%|          | 1307/10000000000 [00:13<29165:17:52, 95.24it/s] 


# クラス分類

In [ ]:
from sklearn.cluster import KMeans, MiniBatchKMeans
from gensim.models.fasttext import load_facebook_model
from tqdm import tqdm
from src.classify.Text2Vec import Text2Vec,texts2classes
import numpy as np


In [ ]:

t2v=Text2Vec(load_facebook_model('../data/model/cc.ja.300.bin'))

In [ ]:
title_vecs=[t2v.text2vec(i) for i in tqdm(cleaned_text)]

100%|██████████| 40004/40004 [01:38<00:00, 406.10it/s] 


In [ ]:
title_vecs=np.array(title_vecs)
# k-meansクラスタリング
print("clustering...")
kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=1).fit(title_vecs)


# 各データポイントが割り当てられたクラスタのインデックスを取得
labels = kmeans.labels_

# 各クラスタに含まれるデータポイントの数を計算
cluster_counts = dict((i, list(labels).count(i)) for i in range(n_clusters))

print(cluster_counts)

clustering...
{0: 1079, 1: 387, 2: 3748, 3: 1037, 4: 2366, 5: 1141, 6: 3802, 7: 3463, 8: 2417, 9: 1391, 10: 2700, 11: 1630, 12: 1670, 13: 2430, 14: 1827, 15: 1016, 16: 963, 17: 1912, 18: 1521, 19: 3504}


In [ ]:
from sklearn.metrics import pairwise_distances_argmin_min
from sklearn.metrics import pairwise_distances
# クラスタのセントロイドを取得
# 仮に title_vecs がデータセットの特徴ベクトル、cleaned_text がそれに対応するクリーンなテキストデータを表すとします。
# n_clusters には、生成したいクラスタの数を指定します。

centroids = kmeans.cluster_centers_

# 各クラスタに属するデータポイントのインデックスを取得
cluster_indices = {i: np.where(kmeans.labels_ == i)[0] for i in range(n_clusters)}

for cluster_id, indices in cluster_indices.items():
    # クラスタ内のデータポイントとセントロイドとの距離を計算
    distances = pairwise_distances(title_vecs[indices], [centroids[cluster_id]])
    # 距離が最も小さい上位5つのデータポイントを選択
    closest_indices = np.argsort(distances[:, 0])[:5]
    
    print(f"Cluster {cluster_id}:")
    for idx in closest_indices:
        # クラスタ内で選択されたデータポイントの実際のインデックス
        actual_idx = indices[idx]
        # テキストが長い場合は100文字に制限し、改行を削除
        print(f"  - {cleaned_text[actual_idx].replace('\n', '')[:100]}")
    print("\n")

Cluster 0:
  - 2011年08月-わたしはここにいます人生を諦めることを覚えようかと思うぐらい(笑)まぁ特に何があったわけではありません。いつものことです。地下鉄メトロの旅以降、写真を取っていない自分がいます。もう3
  - カリスマだんな:So-netブログカリスマだんなよめさんを幸せに、子供を立派にできる男をめざします帝王学その3[子供のこと]能力という面でもうひとつ。僕らが学ぶということで重要になってくるのが、どうや
  - <<お盆に海に行くin福井|TOP|ちょっとうれしい出来事>>最近は忙しくてブログの更新も滞りがちなのだが、以前ナンパしまくっていた時よりなぜかアクセス数が増え続けている。反比例するかのように。まぁ、
  - 誰も知らぬパンドラの匣のロマネスクー迷語殻蛹JISEI録ー:2006年4月\n僕は意識が朦朧として“倒れて"いた。気分はむかむかとした悪いものというより、むしろ心地よさを感じていた。そんな僕の肩から上
  - 協会にお願いしたいこと-30 07:18]リズムの違う国と-30 00:13]高校野球じゃないんだから(16):アンジーからの手紙(15):父ちゃん!巻くんだ!(14):もう負けられない!-27 08


Cluster 1:
  - AYA(トレーナー)の食事は?身長や体重は?本や動画も!プレバト陶芸の先生の教室や料金は?岸野和矢の本や動画も!林家三平はつまらなすぎる?不評?姉や嫁の画像は?初代の動画も!プレバト消しゴムはんこの先
  - 宮崎県・川南町でハイエースバン・ハイエースワゴン中古車を高く売りたい高額買取下取り査定する方法。ハイエースバン・ハイエースワゴンを高額買取下取り査定をしてもらう最もいい方法は、車買取一括買取査定を利用
  - *ようこそ!徳島県・つるぎ町の皆様、【ユーカーパック】がオススメ!事故車!トラックの査定あり!売ることを決めた段階で情報を収集しておけば、10万円単位で高額で買い取ってもらうことは困難ではありません。
  - 現地についたら組み立てて遊びに出かけることができる?、昨日の提携/査定額は、複数の会社の中古が一度に届き。なのに金融相場で株が上がると、バイクを猛スピードで駆け抜ける☆便は明らかに、明らかに整備のピア
  - 車査定のシュミレーション!桑折町でおすすめの一括車査定

In [ ]:
import joblib
#save model
joblib.dump(kmeans, "../data/model/kmeans.pkl")

['../data/model/kmeans.pkl']

In [ ]:
target_texts=cleaned_text[:10]


print(texts2classes(target_texts,t2v,kmeans))

[13 12  7 17 10  7  7  7 16  2]
